# Порівняння bipro та biz (createDate, dateModified)

In [ ]:
import pandas as pd
from custom_func import *

In [2]:
df_P02_006 = pd.read_excel(f"{INPUTS_PATH}/P2/P02_006.xlsx")
df_P02_007 = pd.read_excel(f"{INPUTS_PATH}/P2/P02_007.xlsx")
df_P02_007_dm = pd.read_excel(f"{INPUTS_PATH}/P2/P02_007_dateModified.xlsx")

In [3]:
df_P02_007_dm["dateModified"] = pd.to_datetime(df_P02_007_dm["dateModified"], format="%d.%m.%Y")
df_P02_007_dm = df_P02_007_dm.loc[df_P02_007_dm["dateModified"].between("2020-04-01", "2020-06-30")].copy()

In [4]:
colnames = {
    "Область": "region",
    "Податки на одну особу (p2_01)":'p2_01_raw',
    "Дохід без міжбюдж. трансфертів (p2_02)":'p2_02_raw',
    "Дохід без міжбюдж. трансфертів порівняно з минулим періодом (p2_03)": "p2_03_raw",
    "Капітальні видатки (p2_04)":'p2_04_raw',
    "Плата за землю (p2_05)":'p2_05_raw'
}

df_P02_006 = pd.read_excel(f"{INPUTS_PATH}/P2/P02_006.xlsx").rename(columns=colnames)

In [5]:
normalize_parameter(df_P02_006, "p2_01_raw", "p2_01")
normalize_parameter(df_P02_006, "p2_02_raw", "p2_02")
normalize_parameter(df_P02_006, "p2_03_raw", "p2_03")
normalize_parameter(df_P02_006, "p2_04_raw", "p2_04")
normalize_parameter(df_P02_006, "p2_05_raw", "p2_05")

df_P02_006_fin = df_P02_006.loc[:, df_P02_006.columns.str.contains("region|^p")]

Емпіричні границі: (273.1319492833434, 1419.510256667407)
Коефіцієнти нормалізації: (0.0008723123889895594, -0.23825638318872844)
Параметр p2_01 нормалізовано і додано до таблиці

Емпіричні границі: (689342314.3199999, 6955395883.62)
Коефіцієнти нормалізації: (1.595900815306488e-10, -0.11001219614485493)
Параметр p2_02 нормалізовано і додано до таблиці

Емпіричні границі: (0.624658426803403, 1.168221845737069)
Коефіцієнти нормалізації: (1.8397117340268172, -1.1491914375489523)
Параметр p2_03 нормалізовано і додано до таблиці

Емпіричні границі: (64811628, 1646013700)
Коефіцієнти нормалізації: (6.324302362791237e-10, -0.04098883320967467)
Параметр p2_04 нормалізовано і додано до таблиці

Емпіричні границі: (60392282.4, 729124352.5)
Коефіцієнти нормалізації: (1.4953671951914363e-09, -0.09030863794369715)
Параметр p2_05 нормалізовано і додано до таблиці



---

# Розрахунки на даних, отриманих з BizTenders

In [6]:
df_P02_007_fin = df_P02_007.loc[df_P02_007["status"].eq("complete")].copy()

In [7]:
df_P02_007_fin["Чи відкриті торги?"] = df_P02_007_fin["procurementMethod"].eq("open").astype(int)
open_procur = df_P02_007_fin.loc[df_P02_007_fin['Чи відкриті торги?']==1].groupby('region')["tenderValue"].sum().rename('open')
all_procur = df_P02_007_fin.groupby('region')["tenderValue"].sum().rename('all')
df_P02_007_fin = pd.concat([open_procur, all_procur], axis=1).reset_index()
df_P02_007_fin["p2_06_raw"] = df_P02_007_fin["open"] / df_P02_007_fin["all"]

In [8]:
df_P02_007_fin_biz = df_P02_007_fin.copy()

In [9]:
df_P02_007_fin = df_P02_007_dm.loc[df_P02_007_dm["status"].eq("complete")].copy()
df_P02_007_fin["Чи відкриті торги?"] = df_P02_007_fin["procurementMethod"].eq("open").astype(int)
open_procur = df_P02_007_fin.loc[df_P02_007_fin['Чи відкриті торги?']==1].groupby('region')["tenderValue"].sum().rename('open')
all_procur = df_P02_007_fin.groupby('region')["tenderValue"].sum().rename('all')
df_P02_007_fin = pd.concat([open_procur, all_procur], axis=1).reset_index()
df_P02_007_fin["p2_06_raw"] = df_P02_007_fin["open"] / df_P02_007_fin["all"]
df_P02_007_fin_biz_datemodified = df_P02_007_fin.copy()

---

# Розрахунки на даних bi.prozorro

In [10]:
df = pd.read_excel("./../v0.5/inputs/P2/P02_007_prozorro.xlsx")

open_procedures = ["Відкриті торги", "Відкриті торги із публікацією англ. мовою"]
df["Чи відкриті торги?"] = df["Процедура закупівлі"].isin(open_procedures).astype(int)

df = df.loc[df["Статуси процедур"].eq("Завершена закупівля")].copy()
open_procur = df.loc[df['Чи відкриті торги?']==1].groupby('region')["Очікувана вартість"].sum().rename('open')
all_procur = df.groupby('region')["Очікувана вартість"].sum().rename('all')
df_P02_007_fin = pd.concat([open_procur, all_procur], axis=1).reset_index()
df_P02_007_fin["p2_06_raw"] = df_P02_007_fin["open"] / df_P02_007_fin["all"]

# normalize_parameter(df_P02_007_fin, "p2_06_raw", "p2_06", min_bound=0)

In [11]:
df_P02_007_fin_bipro = df_P02_007_fin.copy()

---

# Розрахунки даних за OB

---

In [12]:
res = pd.merge(
    df_P02_007_fin_bipro.rename(columns={"p2_06_raw": "p2_06_raw_biprozorro"}).drop(["open", "all"], 1),
    df_P02_007_fin_biz.rename(columns={"p2_06_raw": "p2_06_raw_bizTenders_createDate"}).drop(["open", "all"], 1),
    on="region"
).merge(df_P02_007_fin_biz_datemodified.rename(columns={"p2_06_raw": "p2_06_raw_bizTenders_dateModified"}).drop(["open", "all"], 1),on="region")

In [13]:
from tabulate import tabulate

In [25]:
print(tabulate(res.round(2).set_index("region"), tablefmt="pipe", headers="keys"))

| region            |   p2_06_raw_biprozorro |   p2_06_raw_bizTenders_createDate |   p2_06_raw_bizTenders_dateModified |
|:------------------|-----------------------:|----------------------------------:|------------------------------------:|
| Івано-Франківська |                   0.59 |                              0.52 |                                0.84 |
| Волинська         |                   0.48 |                              0.46 |                                0.56 |
| Вінницька         |                   0.82 |                              0.87 |                                0.94 |
| Дніпропетровська  |                   0.47 |                              0.55 |                                0.63 |
| Донецька          |                   0.75 |                              0.78 |                                0.87 |
| Житомирська       |                   0.85 |                              0.88 |                                0.92 |
| Закарпатська      |           